In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [44]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import polars as pl
from xgboost import XGBClassifier

In [4]:
train = pl.read_csv("./playground-series-s4e7/train.csv")
target = pl.read_csv("./playground-series-s4e7/test.csv")

In [9]:
train = train.drop("id")

In [10]:
target = target.drop("id")

In [13]:
category_columns = [col for col, dtype in zip(train.columns, train.dtypes) if dtype == pl.Utf8]

In [18]:
def label_encoding(df, category_columns):

    label_encoders = {col: LabelEncoder() for col in category_columns}
    
    encoded_columns = []
    
    for col in category_columns:
        encoded_col = label_encoders[col].fit_transform(df[col].to_list())
        encoded_columns.append(pl.Series(name=col, values=encoded_col))
        
    df = df.with_columns(encoded_columns)
    
    return df

train = label_encoding(train, category_columns)

shape: (11_504_798, 11)
┌────────┬─────┬─────────────────┬─────────────┬───┬────────────────┬──────────────────────┬─────────┬──────────┐
│ Gender ┆ Age ┆ Driving_License ┆ Region_Code ┆ … ┆ Annual_Premium ┆ Policy_Sales_Channel ┆ Vintage ┆ Response │
│ ---    ┆ --- ┆ ---             ┆ ---         ┆   ┆ ---            ┆ ---                  ┆ ---     ┆ ---      │
│ i64    ┆ i64 ┆ i64             ┆ f64         ┆   ┆ f64            ┆ f64                  ┆ i64     ┆ i64      │
╞════════╪═════╪═════════════════╪═════════════╪═══╪════════════════╪══════════════════════╪═════════╪══════════╡
│ 1      ┆ 21  ┆ 1               ┆ 35.0        ┆ … ┆ 65101.0        ┆ 124.0                ┆ 187     ┆ 0        │
│ 1      ┆ 43  ┆ 1               ┆ 28.0        ┆ … ┆ 58911.0        ┆ 26.0                 ┆ 288     ┆ 1        │
│ 0      ┆ 25  ┆ 1               ┆ 14.0        ┆ … ┆ 38043.0        ┆ 152.0                ┆ 254     ┆ 0        │
│ 0      ┆ 35  ┆ 1               ┆ 1.0         ┆ … ┆ 2630.0     

In [19]:
train

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64,i64
1,21,1,35.0,0,0,1,65101.0,124.0,187,0
1,43,1,28.0,0,2,1,58911.0,26.0,288,1
0,25,1,14.0,1,1,0,38043.0,152.0,254,0
0,35,1,1.0,0,0,1,2630.0,156.0,76,0
0,36,1,15.0,1,0,0,31951.0,152.0,294,0
…,…,…,…,…,…,…,…,…,…,…
1,48,1,6.0,0,0,1,27412.0,26.0,218,0
0,26,1,36.0,0,1,1,29509.0,152.0,115,1
0,29,1,32.0,1,1,0,2630.0,152.0,189,0


In [20]:
target = label_encoding(target, category_columns)

shape: (7_669_866, 10)
┌────────┬─────┬─────────────────┬─────────────┬───┬────────────────┬────────────────┬──────────────────────┬─────────┐
│ Gender ┆ Age ┆ Driving_License ┆ Region_Code ┆ … ┆ Vehicle_Damage ┆ Annual_Premium ┆ Policy_Sales_Channel ┆ Vintage │
│ ---    ┆ --- ┆ ---             ┆ ---         ┆   ┆ ---            ┆ ---            ┆ ---                  ┆ ---     │
│ i64    ┆ i64 ┆ i64             ┆ f64         ┆   ┆ i64            ┆ f64            ┆ f64                  ┆ i64     │
╞════════╪═════╪═════════════════╪═════════════╪═══╪════════════════╪════════════════╪══════════════════════╪═════════╡
│ 0      ┆ 20  ┆ 1               ┆ 47.0        ┆ … ┆ 0              ┆ 2630.0         ┆ 160.0                ┆ 228     │
│ 1      ┆ 47  ┆ 1               ┆ 28.0        ┆ … ┆ 1              ┆ 37483.0        ┆ 124.0                ┆ 123     │
│ 1      ┆ 47  ┆ 1               ┆ 43.0        ┆ … ┆ 1              ┆ 2630.0         ┆ 26.0                 ┆ 271     │
│ 0      ┆ 22  ┆ 

In [21]:
target

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64
0,20,1,47.0,0,1,0,2630.0,160.0,228
1,47,1,28.0,0,0,1,37483.0,124.0,123
1,47,1,43.0,0,0,1,2630.0,26.0,271
0,22,1,47.0,1,1,0,24502.0,152.0,115
1,51,1,19.0,0,0,0,34115.0,124.0,148
…,…,…,…,…,…,…,…,…,…
1,57,1,28.0,0,0,1,51661.0,124.0,109
1,28,1,50.0,1,1,0,25651.0,152.0,184
1,47,1,33.0,1,0,0,2630.0,138.0,63


In [25]:
target = target.with_columns(pl.lit(0).cast(pl.Int64).alias('Response'))

In [26]:
df = pl.concat([train, target])

In [27]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64,i64
1,21,1,35.0,0,0,1,65101.0,124.0,187,0
1,43,1,28.0,0,2,1,58911.0,26.0,288,1
0,25,1,14.0,1,1,0,38043.0,152.0,254,0
0,35,1,1.0,0,0,1,2630.0,156.0,76,0
0,36,1,15.0,1,0,0,31951.0,152.0,294,0
…,…,…,…,…,…,…,…,…,…,…
1,57,1,28.0,0,0,1,51661.0,124.0,109,0
1,28,1,50.0,1,1,0,25651.0,152.0,184,0
1,47,1,33.0,1,0,0,2630.0,138.0,63,0


In [30]:
df = df.with_columns([
    pl.col('Gender').cast(pl.Int32),
    pl.col('Region_Code').cast(int),
    pl.col('Vehicle_Age').cast(pl.Int32),
    pl.col('Vehicle_Damage').cast(pl.Int32),
    pl.col('Annual_Premium').cast(int),
    pl.col('Policy_Sales_Channel').cast(int)
])

In [28]:
df = df.with_columns([
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
])

In [31]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
i32,i64,i64,i64,i64,i32,i32,i64,i64,i64,i64,i64,i64,i64,i64
1,21,1,35,0,0,1,65101,124,187,0,0,0,0,0
1,43,1,28,0,2,1,58911,26,288,1,1,1,0,1
0,25,1,14,1,1,0,38043,152,254,0,2,2,1,2
0,35,1,1,0,0,1,2630,156,76,0,3,0,0,3
0,36,1,15,1,0,0,31951,152,294,0,4,3,1,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,57,1,28,0,0,1,51661,124,109,0,8476,0,0,518
1,28,1,50,1,1,0,25651,152,184,0,28152,2,1,463
1,47,1,33,1,0,0,2630,138,63,0,18,3,1,115


In [32]:
train = df[:train.shape[0]].to_pandas()
test = df[train.shape[0]:].to_pandas()

In [33]:
train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,1,21,1,35,0,0,1,65101,124,187,0,0,0,0,0
1,1,43,1,28,0,2,1,58911,26,288,1,1,1,0,1
2,0,25,1,14,1,1,0,38043,152,254,0,2,2,1,2
3,0,35,1,1,0,0,1,2630,156,76,0,3,0,0,3
4,0,36,1,15,1,0,0,31951,152,294,0,4,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,1,48,1,6,0,0,1,27412,26,218,0,5210,0,0,144
11504794,0,26,1,36,0,1,1,29509,152,115,1,23274,4,0,176
11504795,0,29,1,32,1,1,0,2630,152,189,0,18,2,1,456
11504796,0,51,1,28,0,0,1,48443,26,274,1,14121,0,0,124


In [34]:
test

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,0,20,1,47,0,1,0,2630,160,228,0,3,4,2,55
1,1,47,1,28,0,0,1,37483,124,123,0,1211,0,0,230
2,1,47,1,43,0,0,1,2630,26,271,0,3,0,0,392
3,0,22,1,47,1,1,0,24502,152,115,0,7102,2,1,325
4,1,51,1,19,0,0,0,34115,124,148,0,14835,0,2,415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28,0,0,1,51661,124,109,0,8476,0,0,518
7669862,1,28,1,50,1,1,0,25651,152,184,0,28152,2,1,463
7669863,1,47,1,33,1,0,0,2630,138,63,0,18,3,1,115
7669864,1,30,1,28,0,1,1,38866,124,119,0,5806,4,0,247


In [35]:
df = train.copy()
x= df.drop(columns=['Response'],axis=1)
y= df['Response']

In [36]:
target = test.drop(columns=['Response'],axis=1)

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.1,random_state=0,stratify=y)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from lightgbm import LGBMClassifier

In [39]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, pred_proba)
    AU2 = roc_auc_score(y_test, y_pred.astype('float32'))
    
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    print('AUC2 : {:.4f}'.format(AU2))

In [40]:
x_train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
5853872,0,25,1,29,1,1,0,27605,152,171,7622,2,1,234
4394468,1,25,1,47,1,1,0,45282,152,177,1285,2,1,504
10202146,0,64,1,28,0,2,1,34608,26,251,11933,1,0,165
4141629,1,22,1,8,1,1,0,50398,152,216,24490,2,1,268
10688668,0,45,1,35,0,0,1,27670,124,119,10207,0,0,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5010815,1,52,0,28,1,0,0,55125,26,79,64733,3,1,320
3679955,0,24,1,18,0,1,1,28103,152,212,5319,4,0,332
10562844,1,58,1,39,1,0,0,2630,26,63,18,3,1,115
10500569,1,62,1,28,1,0,1,36177,26,111,47901,3,3,232


In [28]:
y_train

5853872     0
4394468     0
10202146    1
4141629     0
10688668    1
           ..
5010815     0
3679955     0
10562844    0
10500569    0
3678888     0
Name: Response, Length: 10354318, dtype: int8

In [49]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
from hyperopt import STATUS_OK

def objective(params):
    xgb_clf = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        min_child_weight=int(params['min_child_weight']),
        use_label_encoder=False,
        n_jobs=-1,
        eval_metric='auc',
        early_stopping_rounds = int(params['early_stopping_rounds']),
        verbosity=1
    )

    
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    roc_auc_scores = []

    for train_idx, val_idx in skf.split(x_train, y_train):
        X_fold_train, X_fold_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        xgb_clf.fit(X_fold_train, y_fold_train, eval_set=[(X_fold_val, y_fold_val)], verbose=True)
        y_pred_proba = xgb_clf.predict_proba(X_fold_val)[:, 1]  # 클래스 확률 예측
        roc_auc = roc_auc_score(y_fold_val, y_pred_proba, multi_class='ovr')  # 다중 클래스 지원
        roc_auc_scores.append(roc_auc)
    
    mean_auc = np.mean(roc_auc_scores)
    return {'loss': -mean_auc, 'status': STATUS_OK}


In [50]:
space = {
    'n_estimators': hp.quniform('n_estimators', 30, 1000, 1),
    'max_depth': hp.quniform('max_depth', 5, 10, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1), 
    'subsample': hp.uniform('subsample', 0.5, 1.0),  
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  
    'scale_pos_weight': hp.uniform('scale_pos_weight', 4, 15),
    'early_stopping_rounds' : 20
}

In [51]:
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5, trials=trials, rstate=np.random.default_rng(42))

  0%|                                                                            | 0/5 [00:00<?, ?trial/s, best loss=?]

C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")



[0]	validation_0-auc:0.85676                                                                                           
[1]	validation_0-auc:0.85887                                                                                           
[2]	validation_0-auc:0.85921                                                                                           
[3]	validation_0-auc:0.85922                                                                                           
[4]	validation_0-auc:0.85914                                                                                           
[5]	validation_0-auc:0.85923                                                                                           
[6]	validation_0-auc:0.86007                                                                                           
[7]	validation_0-auc:0.85993                                                                                           
[8]	validation_0-auc:0.85990            

[68]	validation_0-auc:0.86498                                                                                          
[69]	validation_0-auc:0.86524                                                                                          
[70]	validation_0-auc:0.86521                                                                                          
[71]	validation_0-auc:0.86543                                                                                          
[72]	validation_0-auc:0.86544                                                                                          
[73]	validation_0-auc:0.86541                                                                                          
[74]	validation_0-auc:0.86540                                                                                          
[75]	validation_0-auc:0.86536                                                                                          
[76]	validation_0-auc:0.86535           

[136]	validation_0-auc:0.86629                                                                                         
[137]	validation_0-auc:0.86629                                                                                         
[138]	validation_0-auc:0.86628                                                                                         
[139]	validation_0-auc:0.86639                                                                                         
[140]	validation_0-auc:0.86641                                                                                         
[141]	validation_0-auc:0.86647                                                                                         
[142]	validation_0-auc:0.86646                                                                                         
[143]	validation_0-auc:0.86646                                                                                         
[144]	validation_0-auc:0.86646          

...

[303]	validation_0-auc:0.87758                                                                                         
[304]	validation_0-auc:0.87760                                                                                         
[305]	validation_0-auc:0.87760                                                                                         
[306]	validation_0-auc:0.87761                                                                                         
[307]	validation_0-auc:0.87762                                                                                         
[308]	validation_0-auc:0.87764                                                                                         
[0]	validation_0-auc:0.85352                                                                                           
[1]	validation_0-auc:0.85578                                                                                           
[2]	validation_0-auc:0.85839            

[62]	validation_0-auc:0.86553                                                                                          
[63]	validation_0-auc:0.86550                                                                                          
[64]	validation_0-auc:0.86550                                                                                          
[65]	validation_0-auc:0.86547                                                                                          
[66]	validation_0-auc:0.86578                                                                                          
[67]	validation_0-auc:0.86579                                                                                          
[68]	validation_0-auc:0.86603                                                                                          
[69]	validation_0-auc:0.86609                                                                                          
[70]	validation_0-auc:0.86610           

[130]	validation_0-auc:0.87122                                                                                         
[131]	validation_0-auc:0.87125                                                                                         
[132]	validation_0-auc:0.87128                                                                                         
[133]	validation_0-auc:0.87139                                                                                         
[134]	validation_0-auc:0.87144                                                                                         
[135]	validation_0-auc:0.87145                                                                                         
[136]	validation_0-auc:0.87147                                                                                         
[137]	validation_0-auc:0.87152                                                                                         
[138]	validation_0-auc:0.87160          

[198]	validation_0-auc:0.87528                                                                                         
[199]	validation_0-auc:0.87531                                                                                         
[200]	validation_0-auc:0.87537                                                                                         
[201]	validation_0-auc:0.87544                                                                                         
[202]	validation_0-auc:0.87547                                                                                         
[203]	validation_0-auc:0.87556                                                                                         
[204]	validation_0-auc:0.87557                                                                                         
[205]	validation_0-auc:0.87561                                                                                         
[206]	validation_0-auc:0.87564          

[266]	validation_0-auc:0.87725                                                                                         
[267]	validation_0-auc:0.87727                                                                                         
[268]	validation_0-auc:0.87728                                                                                         
[269]	validation_0-auc:0.87730                                                                                         
[270]	validation_0-auc:0.87730                                                                                         
[271]	validation_0-auc:0.87735                                                                                         
[272]	validation_0-auc:0.87738                                                                                         
[273]	validation_0-auc:0.87739                                                                                         
[274]	validation_0-auc:0.87741          

In [52]:
print("Best parameters found: ", best)

Best parameters found:  {'colsample_bytree': 0.9455746297888614, 'learning_rate': 0.1645541835252844, 'max_depth': 8.0, 'min_child_weight': 2.0, 'n_estimators': 629.0, 'scale_pos_weight': 4.810271279955765, 'subsample': 0.5734813936425138}


In [53]:
best_params = {
    'n_estimators': int(629.0),
    'learning_rate': 0.1645541835252844,
    'min_child_weight': int(2.0),  
    'max_depth': int(8.0),
    'subsample': 0.5734813936425138,
    'colsample_bytree': 0.9455746297888614,
    'scale_pos_weight': 4.810271279955765,
    'n_jobs': -1
}

In [54]:
best_Xgb = XGBClassifier(**best_params,early_stopping_rounds=20,eval_metric='auc')
best_Xgb.fit(x_train, y_train,eval_set=[(x_val, y_val)],verbose=True)

[0]	validation_0-auc:0.85633
[1]	validation_0-auc:0.85793
[2]	validation_0-auc:0.85933
[3]	validation_0-auc:0.85979
[4]	validation_0-auc:0.85994
[5]	validation_0-auc:0.86057
[6]	validation_0-auc:0.86312
[7]	validation_0-auc:0.86323
[8]	validation_0-auc:0.86466
[9]	validation_0-auc:0.86472
[10]	validation_0-auc:0.86472
[11]	validation_0-auc:0.86482
[12]	validation_0-auc:0.86538
[13]	validation_0-auc:0.86564
[14]	validation_0-auc:0.86675
[15]	validation_0-auc:0.86679
[16]	validation_0-auc:0.86690
[17]	validation_0-auc:0.86726
[18]	validation_0-auc:0.86872
[19]	validation_0-auc:0.86976
[20]	validation_0-auc:0.87000
[21]	validation_0-auc:0.87077
[22]	validation_0-auc:0.87094
[23]	validation_0-auc:0.87103
[24]	validation_0-auc:0.87178
[25]	validation_0-auc:0.87182
[26]	validation_0-auc:0.87205
[27]	validation_0-auc:0.87295
[28]	validation_0-auc:0.87306
[29]	validation_0-auc:0.87338
[30]	validation_0-auc:0.87346
[31]	validation_0-auc:0.87377
[32]	validation_0-auc:0.87424
[33]	validation_0-au

[268]	validation_0-auc:0.88392
[269]	validation_0-auc:0.88392
[270]	validation_0-auc:0.88392
[271]	validation_0-auc:0.88394
[272]	validation_0-auc:0.88396
[273]	validation_0-auc:0.88399
[274]	validation_0-auc:0.88400
[275]	validation_0-auc:0.88401
[276]	validation_0-auc:0.88401
[277]	validation_0-auc:0.88401
[278]	validation_0-auc:0.88401
[279]	validation_0-auc:0.88401
[280]	validation_0-auc:0.88401
[281]	validation_0-auc:0.88402
[282]	validation_0-auc:0.88402
[283]	validation_0-auc:0.88403
[284]	validation_0-auc:0.88403
[285]	validation_0-auc:0.88405
[286]	validation_0-auc:0.88405
[287]	validation_0-auc:0.88406
[288]	validation_0-auc:0.88406
[289]	validation_0-auc:0.88413
[290]	validation_0-auc:0.88413
[291]	validation_0-auc:0.88414
[292]	validation_0-auc:0.88416
[293]	validation_0-auc:0.88420
[294]	validation_0-auc:0.88420
[295]	validation_0-auc:0.88420
[296]	validation_0-auc:0.88420
[297]	validation_0-auc:0.88421
[298]	validation_0-auc:0.88421
[299]	validation_0-auc:0.88421
[300]	va

[533]	validation_0-auc:0.88528
[534]	validation_0-auc:0.88528
[535]	validation_0-auc:0.88528
[536]	validation_0-auc:0.88528
[537]	validation_0-auc:0.88527
[538]	validation_0-auc:0.88527
[539]	validation_0-auc:0.88527
[540]	validation_0-auc:0.88527
[541]	validation_0-auc:0.88527
[542]	validation_0-auc:0.88527
[543]	validation_0-auc:0.88527
[544]	validation_0-auc:0.88528
[545]	validation_0-auc:0.88528
[546]	validation_0-auc:0.88528
[547]	validation_0-auc:0.88529
[548]	validation_0-auc:0.88529
[549]	validation_0-auc:0.88529
[550]	validation_0-auc:0.88529
[551]	validation_0-auc:0.88528
[552]	validation_0-auc:0.88528
[553]	validation_0-auc:0.88528
[554]	validation_0-auc:0.88529
[555]	validation_0-auc:0.88529
[556]	validation_0-auc:0.88530
[557]	validation_0-auc:0.88530
[558]	validation_0-auc:0.88531
[559]	validation_0-auc:0.88532
[560]	validation_0-auc:0.88531
[561]	validation_0-auc:0.88532
[562]	validation_0-auc:0.88532
[563]	validation_0-auc:0.88532
[564]	validation_0-auc:0.88533
[565]	va

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9455746297888614, early_stopping_rounds=20,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1645541835252844,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=629, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [55]:
pred = best_Xgb.predict(x_val)
pred_proba = best_Xgb.predict_proba(x_val)[:,1]
get_clf_eval(y_val,pred,pred_proba)


정확도: 0.7787
정밀도: 0.3397
재현율: 0.8470
F1: 0.4849
AUC: 0.8854
AUC2 : 0.8081


In [56]:
a=best_Xgb.predict_proba(target)[:,1]

In [57]:
submit = pd.read_csv("./playground-series-s4e7/sample_submission.csv")
pred = a 
submit["Response"] = pred
submit.to_csv("submission_XGB5.csv", index=False)
submit.head()

,id,Response
0,11504798,0.012422
1,11504799,0.871203
2,11504800,0.599751
3,11504801,0.000379
4,11504802,0.420601
